<a href="https://colab.research.google.com/github/pirate264/Quantum-Image-Processing/blob/main/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===== Reproducibility Control =====
import random, time, tracemalloc
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [4]:
import zipfile, os, cv2

ZIP_PATH = "/content/brainTumor_JPG.zip"
DATA_PATH = "/content/brainTumor_JPG"

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(DATA_PATH)

In [5]:
def load_images(root, size=(256,256)):
    imgs = []
    for r, _, files in os.walk(root):
        for f in files:
            if f.lower().endswith(('.jpg','.png','.jpeg')):
                img = cv2.imread(os.path.join(r,f))
                if img is None: continue
                img = cv2.resize(img, size)
                imgs.append(img)
    return np.array(imgs)

images = load_images(DATA_PATH)
print("Total images:", images.shape)

Total images: (766, 256, 256, 3)


In [6]:
def preprocess(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = g.astype(np.float32)
    g = (g - g.min()) / (g.max() - g.min() + 1e-8)
    return g

images = np.array([preprocess(i) for i in images])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_tmp = train_test_split(images, test_size=0.30, random_state=SEED)
X_val, X_test  = train_test_split(X_tmp,  test_size=0.50, random_state=SEED)

print(X_train.shape, X_val.shape, X_test.shape)

(536, 256, 256) (115, 256, 256) (115, 256, 256)


In [8]:
def add_noise(img, sigma=0.05):
    n = np.random.normal(0, sigma, img.shape)
    return np.clip(img + n, 0, 1)

In [30]:
from skimage.filters import sobel, prewitt, laplace
from sklearn.metrics import precision_score, recall_score, f1_score
from skimage.metrics import structural_similarity as ssim

def compute_metrics(ref, pred):
    ref_b = (ref > 0).astype(np.uint8).flatten()
    pred_b = (pred > 0.5).astype(np.uint8).flatten()

    return (
        precision_score(ref_b, pred_b, zero_division=0),
        recall_score(ref_b, pred_b, zero_division=0),
        f1_score(ref_b, pred_b, zero_division=0),
        ssim(ref, pred, data_range=1.0)
    )

In [10]:
def profile(model_fn, img):
    tracemalloc.start()
    t0 = time.time()
    out = model_fn(img)
    t1 = time.time()
    mem = tracemalloc.get_traced_memory()[1] / 1024**2
    tracemalloc.stop()
    return out, t1 - t0, mem

In [11]:
from scipy.stats import t

def stats(values):
    mean = np.mean(values)
    std  = np.std(values)
    ci   = t.ppf(0.975, len(values)-1) * std / np.sqrt(len(values))
    return mean, std, mean-ci, mean+ci

In [13]:
def model_2d_qsna(img):
    grad = sobel(img)
    norm = grad / (np.linalg.norm(grad) + 1e-8)
    return norm

In [14]:
def model_frqi(img):
    theta = img * np.pi
    return np.abs(np.sin(theta))

In [15]:
def model_efrqi(img):
    theta = img * np.pi
    return np.abs(np.sin(theta)) * sobel(img)

In [16]:
def model_gqir(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    return np.sqrt(gx**2 + gy**2)

In [17]:
def model_iqir(img):
    return prewitt(img)

In [18]:
def model_mcqi(img):
    stacked = np.stack([img, sobel(img), prewitt(img)], axis=0)
    return np.mean(stacked, axis=0)

In [19]:
def model_nass(img):
    aux = laplace(img)
    return np.abs(img * aux)

In [20]:
def model_neqr(img):
    quant = np.round(img * 255) / 255
    return sobel(quant)

In [21]:
def model_piqed(img):
    grad = sobel(img)
    phase = np.pi * grad
    return (1 - np.cos(phase)) / 2

In [22]:
MODELS = {
    "2D-QSNA": model_2d_qsna,
    "EFRQI": model_efrqi,
    "FRQI": model_frqi,
    "GQIR": model_gqir,
    "IQIR": model_iqir,
    "MCQI": model_mcqi,
    "NASS": model_nass,
    "NEQR": model_neqr,
    "PIQED": model_piqed
}

In [32]:
RESULTS = {}
RUNS = 5

img = X_test[0]
ref = sobel(img)

for name, model in MODELS.items():
    P,R,F,S,RT,M = [],[],[],[],[],[]

    for _ in range(RUNS):
        noisy = add_noise(img).astype(np.float32)
        pred, rt, mem = profile(model, noisy)
        p,r,f,s = compute_metrics(ref, pred)

        P.append(p); R.append(r); F.append(f)
        S.append(s); RT.append(rt); M.append(mem)

    RESULTS[name] = {
        "Precision": stats(P),
        "Recall": stats(R),
        "F1": stats(F),
        "SSIM": stats(S),
        "Runtime": stats(RT),
        "Memory": stats(M)
    }

In [33]:
import pandas as pd

rows = []
for model, vals in RESULTS.items():
    for metric, v in vals.items():
        rows.append([
            model, metric,
            f"{v[0]:.4f} ± {v[1]:.4f}",
            f"[{v[2]:.4f}, {v[3]:.4f}]"
        ])

df = pd.DataFrame(rows, columns=["Model","Metric","Mean ± Std","95% CI"])
df

,Model,Metric,Mean ± Std,95% CI
0,2D-QSNA,Precision,0.0000 ± 0.0000,"[0.0000, 0.0000]"
1,2D-QSNA,Recall,0.0000 ± 0.0000,"[0.0000, 0.0000]"
2,2D-QSNA,F1,0.0000 ± 0.0000,"[0.0000, 0.0000]"
3,2D-QSNA,SSIM,0.3835 ± 0.0003,"[0.3832, 0.3839]"
4,2D-QSNA,Runtime,0.0053 ± 0.0019,"[0.0030, 0.0077]"
5,2D-QSNA,Memory,0.7513 ± 0.0001,"[0.7512, 0.7514]"
6,EFRQI,Precision,1.0000 ± 0.0000,"[1.0000, 1.0000]"
7,EFRQI,Recall,0.0003 ± 0.0000,"[0.0002, 0.0003]"
8,EFRQI,F1,0.0006 ± 0.0001,"[0.0005, 0.0006]"
9,EFRQI,SSIM,0.7134 ± 0.0012,"[0.7119, 0.7149]"


In [34]:
# Convert long-format results to wide table
pivot_df = df.pivot(
    index="Model",
    columns="Metric",
    values="Mean ± Std"
)

pivot_df

Metric,F1,Memory,Precision,Recall,Runtime,SSIM
Model,,,,,,
2D-QSNA,0.0000 ± 0.0000,0.7513 ± 0.0001,0.0000 ± 0.0000,0.0000 ± 0.0000,0.0053 ± 0.0019,0.3835 ± 0.0003
EFRQI,0.0006 ± 0.0001,1.2514 ± 0.0000,1.0000 ± 0.0000,0.0003 ± 0.0000,0.0063 ± 0.0019,0.7134 ± 0.0012
FRQI,0.7730 ± 0.0008,0.7503 ± 0.0000,0.9969 ± 0.0001,0.6312 ± 0.0011,0.0004 ± 0.0000,0.0310 ± 0.0002
GQIR,0.3273 ± 0.0004,1.0004 ± 0.0000,0.9997 ± 0.0001,0.1957 ± 0.0003,0.0009 ± 0.0009,0.0442 ± 0.0003
IQIR,0.0007 ± 0.0002,0.7512 ± 0.0000,1.0000 ± 0.0000,0.0004 ± 0.0001,0.0056 ± 0.0026,0.6020 ± 0.0012
MCQI,0.0022 ± 0.0001,1.2513 ± 0.0000,1.0000 ± 0.0000,0.0011 ± 0.0001,0.0126 ± 0.0046,0.3795 ± 0.0013
NASS,0.0031 ± 0.0003,0.7505 ± 0.0000,1.0000 ± 0.0000,0.0016 ± 0.0001,0.0034 ± 0.0016,0.3198 ± 0.0040
NEQR,0.0011 ± 0.0002,1.0013 ± 0.0000,1.0000 ± 0.0000,0.0006 ± 0.0001,0.0042 ± 0.0001,0.5837 ± 0.0008
PIQED,0.0012 ± 0.0001,1.0008 ± 0.0000,1.0000 ± 0.0000,0.0006 ± 0.0000,0.0043 ± 0.0001,0.6587 ± 0.0015


In [35]:
def robustness_score(model, img, noise_sigma=0.05):
    clean = model(img)
    noisy = model(add_noise(img, sigma=noise_sigma))

    ssim_clean = ssim(img, clean, data_range=1.0)
    ssim_noisy = ssim(img, noisy, data_range=1.0)

    return 1 - abs(ssim_clean - ssim_noisy) / (ssim_clean + 1e-8)

In [39]:
pivot_df['Runtime']

,Runtime
Model,
2D-QSNA,0.0053 ± 0.0019
EFRQI,0.0063 ± 0.0019
FRQI,0.0004 ± 0.0000
GQIR,0.0009 ± 0.0009
IQIR,0.0056 ± 0.0026
MCQI,0.0126 ± 0.0046
NASS,0.0034 ± 0.0016
NEQR,0.0042 ± 0.0001
PIQED,0.0043 ± 0.0001


In [60]:
def scalability_test(model_fn, img, runs=3):
    import numpy as np, cv2

    sizes = [128, 256, 512]
    pixels = np.array([s*s for s in sizes], dtype=np.float32)
    times = []

    for s in sizes:
        resized = cv2.resize(img, (s, s))
        run_times = []

        for _ in range(runs):
            _, t, _ = profile(model_fn, resized)
            run_times.append(max(t, 1e-6))

        times.append(np.mean(run_times))

    log_pixels = np.log(pixels)
    log_times = np.log(times)

    slope, _ = np.polyfit(log_pixels, log_times, 1)
    return slope

In [61]:
def scalability_label(factor):
    if factor < 4:
        return "High"
    elif factor < 10:
        return "Medium"
    else:
        return "Low"

In [62]:
def scalability_full_dataset(model_fn, X_test, runs=3, max_samples=None):
    """
    Computes scalability over the full test dataset.
    Optionally limit samples for speed.
    """
    SC = []

    if max_samples is None:
        images = X_test
    else:
        images = X_test[:max_samples]

    for img in images:
        sc = scalability_test(model_fn, img, runs=runs)
        SC.append(sc)

    SC = np.array(SC)

    mean_sc = np.mean(SC)
    std_sc  = np.std(SC)

    # 95% confidence interval
    from scipy.stats import t
    ci = t.ppf(0.975, len(SC)-1) * std_sc / np.sqrt(len(SC))

    return {
        "mean": mean_sc,
        "std": std_sc,
        "ci_low": mean_sc - ci,
        "ci_high": mean_sc + ci,
        "label": scalability_label(mean_sc)
    }

In [63]:
SCALABILITY_RESULTS = {}

for name, model_fn in MODELS.items():
    res = scalability_full_dataset(
        model_fn,
        X_test,
        runs=3,
        max_samples=20  # set None for full dataset
    )

    SCALABILITY_RESULTS[name] = res

    print(
        f"{name}: {res['mean']:.4f} ± {res['std']:.4f} "
        f"[{res['ci_low']:.4f}, {res['ci_high']:.4f}] → {res['label']}"
    )

2D-QSNA: 0.7615 ± 0.1098 [0.7101, 0.8129] → High
EFRQI: 0.7948 ± 0.0867 [0.7543, 0.8354] → High
FRQI: 0.9495 ± 0.0563 [0.9231, 0.9758] → High
GQIR: 0.7940 ± 0.0265 [0.7816, 0.8064] → High
IQIR: 0.7814 ± 0.0392 [0.7631, 0.7997] → High
MCQI: 0.8500 ± 0.0397 [0.8314, 0.8686] → High
NASS: 0.6027 ± 0.0290 [0.5891, 0.6162] → High
NEQR: 0.7918 ± 0.0426 [0.7719, 0.8118] → High
PIQED: 0.7722 ± 0.0405 [0.7532, 0.7912] → High


In [64]:
for name in RESULTS:
    RESULTS[name]["Scalability"] = SCALABILITY_RESULTS[name]

In [66]:
import pandas as pd

rows = []
for model, res in SCALABILITY_RESULTS.items():
    rows.append([
        model,
        f"{res['mean']:.4f} ± {res['std']:.4f}",
        f"[{res['ci_low']:.4f}, {res['ci_high']:.4f}]",
        res['label']
    ])

scalability_df = pd.DataFrame(
    rows,
    columns=["Model", "Scalability Index", "95% CI", "Category"]
)

scalability_df

,Model,Scalability Index,95% CI,Category
0,2D-QSNA,0.7615 ± 0.1098,"[0.7101, 0.8129]",High
1,EFRQI,0.7948 ± 0.0867,"[0.7543, 0.8354]",High
2,FRQI,0.9495 ± 0.0563,"[0.9231, 0.9758]",High
3,GQIR,0.7940 ± 0.0265,"[0.7816, 0.8064]",High
4,IQIR,0.7814 ± 0.0392,"[0.7631, 0.7997]",High
5,MCQI,0.8500 ± 0.0397,"[0.8314, 0.8686]",High
6,NASS,0.6027 ± 0.0290,"[0.5891, 0.6162]",High
7,NEQR,0.7918 ± 0.0426,"[0.7719, 0.8118]",High
8,PIQED,0.7722 ± 0.0405,"[0.7532, 0.7912]",High
